In [198]:
from requests import get
from bs4 import BeautifulSoup as soup 
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from time import sleep
import pandas as pd
import yfinance as yf
from datetime import datetime,timedelta

#### This piece of code has a high chance of not running. I have found that running it 2-3 (i mean 10-15 actually) times will give the required output
The underlying problem is that the NSE website is not always dependable and the javascript doesnt always load. This basically means that the webdriver is scraping partially rendered websites, not becuase of lack of waiting period given but some irreguarity from the NSE's end.  

In [72]:
edge_driver_path = "C:\\Users\\kidam\\Downloads\\msedgedriver.exe"

edge_options = Options()
    
# Enable JavaScript
edge_options.add_argument("--enable-javascript")

# edge_options.add_argument("--start-maximized")
# edge_options.add_argument("--disable-extensions")

service = Service(executable_path=edge_driver_path)
driver = webdriver.Edge(service=service)
driver.get("https://www.nseindia.com/market-data/live-equity-market?symbol=NIFTY%2050")

WebDriverWait(driver, 10).until(
  lambda d: d.execute_script("return jQuery.active") == 0
  if d.execute_script("return typeof jQuery != 'undefined'")
  else True
)

# # Wait for Angular to be complete if present
# WebDriverWait(driver, 10).until(
#   lambda d: d.execute_script("return window.getAllAngularTestabilities().findIndex(x=>!x.isStable()) === -1")
#   if d.execute_script("return typeof angular !== 'undefined'")
#   else True
# )

# Wait for document to be ready
# WebDriverWait(driver, 10).until(
#   lambda d: d.execute_script("return document.readyState") == "complete"
# )

# Additional small delay to ensure full rendering
sleep(10)

page_source = driver.page_source

##### Only run this code if the above code doesnt work even after multiple tries

In [98]:
import pickle
f = open("data.bin","rb")
page_source = pickle.load(f)

Containing (Y)PE, EPS, (Y)52 Week High/Low, (Y)Lower and Upper Circuit, (Y)LTP, and (Y)Market Cap   
 , (Y)Volume, (Y)% Change and convert to DataFrame.


In [74]:
bs = soup(page_source, "lxml")
l = bs.find_all("tr")

In [189]:
def get_parameters(symbol):
  edge_driver_path = "C:\\Users\\kidam\\Downloads\\msedgedriver.exe"

  edge_options = Options()

  # Enable JavaScript
  edge_options.add_argument("--enable-javascript")

  service = Service(executable_path=edge_driver_path)
  driver = webdriver.Edge(service=service)
  driver.get(f"https://www.nseindia.com/get-quotes/equity?symbol={symbol}")

  WebDriverWait(driver, 10).until(
    lambda d: d.execute_script("return jQuery.active") == 0
    if d.execute_script("return typeof jQuery != 'undefined'")
    else True
  )
  # Additional small delay to ensure full rendering
  sleep(10)

  new_source = driver.page_source
  driver.close()
  new_source = soup(new_source,"lxml")
  PE = float(new_source.find("table",attrs={"id":"securities_info_table"}).findAll("tr")[5].findAll("td")[1].text)
  UB = float(new_source.findAll("table",attrs = {"role":"presentation"})[1].find_all("tr")[3].find("span").text)
  LB = float(new_source.findAll("table",attrs = {"role":"presentation"})[1].find_all("tr")[4].find("span").text)
  # EPS = new_source.find("table",attrs = {"id":"topFinancialResultsTable"}).find("tr").find_all("td")[-1]
  
  return PE,UB,LB
get_parameters("WIPRO")

  

(27.3, 340.0, 278.2)

In [121]:
edge_driver_path = "C:\\Users\\kidam\\Downloads\\msedgedriver.exe"

edge_options = Options()
# Enable JavaScript
edge_options.add_argument("--enable-javascript")
service = Service(executable_path=edge_driver_path)
driver = webdriver.Edge(service=service)
driver.get(f"https://www.nseindia.com/get-quotes/equity?symbol=BHARTIARTL")
WebDriverWait(driver, 10).until(
  lambda d: d.execute_script("return jQuery.active") == 0
  if d.execute_script("return typeof jQuery != 'undefined'")
  else True
)
# Additional small delay to ensure full rendering
sleep(10)
new_source = driver.page_source

new_source = soup(new_source,"lxml")
PE = float(new_source.find("table",attrs={"id":"securities_info_table"}).findAll("tr")[5].findAll("td")[1].text)
UB = float(new_source.findAll("table",attrs = {"role":"presentation"})[1].find_all("tr")[3].find("span").text)
LB = float(new_source.findAll("table",attrs = {"role":"presentation"})[1].find_all("tr")[4].find("span").text)

AttributeError: 'NoneType' object has no attribute 'find'

'1,449.50'

In [176]:
stocks = []
for i in range(5,205,4):
  stocks.append(list(map(lambda x : x.text, l[i].findAll("td")))[:13])
stocks

[['BHARTIARTL',
  '1,618.00',
  '1,685.00',
  '1,606.80',
  '1,610.55',
  '1,682.00',
  '-',
  '71.45',
  '4.44',
  '1,29,00,707',
  '2,134.71',
  '1,779.00',
  '960.00'],
 ['ITC',
  '460.00',
  '474.40',
  '451.65',
  '460.60',
  '471.00',
  '-',
  '10.40',
  '2.26',
  '3,17,53,858',
  '1,474.84',
  '528.50',
  '399.35'],
 ['KOTAKBANK',
  '1,758.35',
  '1,809.00',
  '1,748.05',
  '1,768.70',
  '1,804.90',
  '-',
  '36.20',
  '2.05',
  '38,93,531',
  '692.05',
  '1,942.00',
  '1,543.85'],
 ['HINDUNILVR',
  '2,344.95',
  '2,394.55',
  '2,333.45',
  '2,344.95',
  '2,391.25',
  '-',
  '46.30',
  '1.97',
  '22,60,902',
  '535.98',
  '3,035.00',
  '2,172.05'],
 ['ULTRACEMCO',
  '11,800.00',
  '12,118.50',
  '11,730.00',
  '11,856.95',
  '12,082.35',
  '-',
  '225.40',
  '1.90',
  '4,08,880',
  '489.50',
  '12,138.00',
  '9,250.00'],
 ['TITAN',
  '3,420.00',
  '3,518.00',
  '3,380.25',
  '3,445.75',
  '3,511.00',
  '-',
  '65.25',
  '1.89',
  '12,42,351',
  '429.83',
  '3,886.95',
  '3,055.6

In [177]:
df = pd.DataFrame(stocks,columns=[i for i in list(map(lambda x : x.text,l[0].findAll("span")))[:-2] if not i.startswith("(")])
df

,Symbol,Open,High,Low,PREV. CLOSE,LTP,Indicative CLose,chng,%Chng,Volume,Value,52W H,52W L
0,BHARTIARTL,"1,618.00","1,685.00","1,606.80","1,610.55","1,682.00",-,71.45,4.44,"1,29,00,707","2,134.71","1,779.00",960.00
1,ITC,460.00,474.40,451.65,460.60,471.00,-,10.40,2.26,"3,17,53,858","1,474.84",528.50,399.35
2,KOTAKBANK,"1,758.35","1,809.00","1,748.05","1,768.70","1,804.90",-,36.20,2.05,"38,93,531",692.05,"1,942.00","1,543.85"
3,HINDUNILVR,"2,344.95","2,394.55","2,333.45","2,344.95","2,391.25",-,46.30,1.97,"22,60,902",535.98,"3,035.00","2,172.05"
4,ULTRACEMCO,"11,800.00","12,118.50","11,730.00","11,856.95","12,082.35",-,225.40,1.90,"4,08,880",489.50,"12,138.00","9,250.00"
5,TITAN,"3,420.00","3,518.00","3,380.25","3,445.75","3,511.00",-,65.25,1.89,"12,42,351",429.83,"3,886.95","3,055.65"
6,POWERGRID,330.65,334.30,325.05,329.20,334.10,-,4.90,1.49,"1,49,73,536",494.97,366.25,222.70
7,HCLTECH,"1,939.00","1,975.00","1,912.35","1,936.20","1,965.00",-,28.80,1.49,"25,69,918",502.41,"1,975.00","1,235.00"
8,NESTLEIND,"2,225.00","2,257.85","2,208.25","2,224.05","2,255.00",-,30.95,1.39,"5,54,661",124.14,"2,778.00","2,168.70"
9,GRASIM,"2,641.00","2,700.00","2,620.50","2,660.05","2,696.05",-,36.00,1.35,"4,12,267",110.14,"2,877.75","2,016.55"


In [220]:
t = df['Symbol'].apply(get_parameters)

IndexError: list index out of range

In [ ]:
t = sum(t,[])
df['PE'], df['UB'], df['LB'] = t[range(0,51,3),range(1,51,3),range(2,51,3)]

In [206]:
def get_returns(ticker):
  stock = yf.Ticker(ticker)
  end_date = datetime.now()
  periods = {
    '6_months': end_date - timedelta(days=180),
    '1_year': end_date - timedelta(days=365),
    '5_years': end_date - timedelta(days=365*5)
  }

  # Store returns
  returns = {}
  for period_name, start_date in periods.items():
    # Fetch historical data
    hist = stock.history(start=start_date, end=end_date)

    # Calculate return percentage
    if not hist.empty:
      start_price = hist['Close'].iloc[0]
      end_price = hist['Close'].iloc[-1]

      return_percentage = ((end_price - start_price) / start_price) * 100
      returns[period_name] = round(return_percentage, 2)
    else:
      returns[period_name] = None
  return returns
  

In [217]:
t = (df['Symbol'] + ".NS").apply(get_returns)
returns_data = [list(),list(),list()]
for i in t:
  returns_data[0].append(i['6_months'])
  returns_data[1].append(i['1_year'])
  returns_data[2].append(i['5_years'])
df['6 Months'],df['1 Year'],df['5 Years'] = returns_data


In [216]:
df

,Symbol,Open,High,Low,PREV. CLOSE,LTP,Indicative CLose,chng,%Chng,Volume,Value,52W H,52W L,PE,UB,LB,6 Months,1 Year,5 Years
0,BHARTIARTL,"1,618.00","1,685.00","1,606.80","1,610.55","1,682.00",-,71.45,4.44,"1,29,00,707","2,134.71","1,779.00",960.00,1,2,3,21.46,70.78,298.07
1,ITC,460.00,474.40,451.65,460.60,471.00,-,10.40,2.26,"3,17,53,858","1,474.84",528.50,399.35,1,2,3,10.94,7.46,134.40
2,KOTAKBANK,"1,758.35","1,809.00","1,748.05","1,768.70","1,804.90",-,36.20,2.05,"38,93,531",692.05,"1,942.00","1,543.85",1,2,3,3.49,-2.13,5.03
3,HINDUNILVR,"2,344.95","2,394.55","2,333.45","2,344.95","2,391.25",-,46.30,1.97,"22,60,902",535.98,"3,035.00","2,172.05",1,2,3,-1.98,-4.09,34.13
4,ULTRACEMCO,"11,800.00","12,118.50","11,730.00","11,856.95","12,082.35",-,225.40,1.90,"4,08,880",489.50,"12,138.00","9,250.00",1,2,3,10.55,21.92,204.52
5,TITAN,"3,420.00","3,518.00","3,380.25","3,445.75","3,511.00",-,65.25,1.89,"12,42,351",429.83,"3,886.95","3,055.65",1,2,3,1.67,-2.74,209.89
6,POWERGRID,330.65,334.30,325.05,329.20,334.10,-,4.90,1.49,"1,49,73,536",494.97,366.25,222.70,1,2,3,4.33,49.74,326.63
7,HCLTECH,"1,939.00","1,975.00","1,912.35","1,936.20","1,965.00",-,28.80,1.49,"25,69,918",502.41,"1,975.00","1,235.00",1,2,3,38.11,35.57,310.70
8,NESTLEIND,"2,225.00","2,257.85","2,208.25","2,224.05","2,255.00",-,30.95,1.39,"5,54,661",124.14,"2,778.00","2,168.70",1,2,3,-10.40,-6.81,67.23
9,GRASIM,"2,641.00","2,700.00","2,620.50","2,660.05","2,696.05",-,36.00,1.35,"4,12,267",110.14,"2,877.75","2,016.55",1,2,3,10.46,28.63,266.07
